# 튜토리얼 6 - 포트폴리오 백테스트

본 튜토리얼에서는 개별 주식이 아닌 복수의 주식 즉, 포트폴리오(portfolio)을 대상으로 백테스트하는 법을 설명합니다.

In [1]:
import kquant as kq

## 6.1 백테스트용 입력정보 준비

포트폴리오 백테스트를 하기 위해서는 날짜별 주식 주문 정보가 담긴 pandas 데이터프레임 `df_order`을 준비해야 합니다. 
`df_order`은 다음과 같은 3개의 열을 가져야 합니다.

- `DATE`: 주문하는 주식의 날짜 정보 리스트. 중복된 날짜가 있으면 안되고 pandas `to_datetime` 함수로 변환가능한 `"2023-01-01` 형식 등의 - `SYMBOL`: 주문하는 주식의 종목 단축코드. 모든 행에 대해 동일한 값을 가져야 함
문자열 
- `ORDER`: 주문하는 주식의 수량 리스트. 정수만 가능하며 양수인 경우 매수, 음수인 경우 매도로 처리함


예를 들어 다음과 같이 2023년 1월 2일에 삼성전자 주식 10주와 SK하이닉스 주식 20주를 매수하고 11일에 삼성전자를 전량 매도, 18일에 SK하이닉스를 전량 매도하는 주문정보를 만들 수 있습니다.

In [2]:
import pandas as pd

df_order = pd.DataFrame({
    "DATE": ["2023-01-02", "2023-01-02", "2023-01-11", "2023-01-18"],
    "SYMBOL": ["005930", "000660", "005930", "000660"],
    "ORDER": [10, 20, -10, -20]
})

df_order


,DATE,SYMBOL,ORDER
0,2023-01-02,005930,10
1,2023-01-02,000660,20
2,2023-01-11,005930,-10
3,2023-01-18,000660,-20


## 6.2 백테스트 실시

포트폴리오 주문 정보 데이터프레임이 준비되면 `backtest_stock_port_daily` 함수를 사용하여 포트폴리오 매매 백테스트를 할 수 있습니다. 

`backtest_stock_port_daily` 함수는 기본적으로 다음과 같은 입력 인수를 받습니다.

- `order`: 날짜별 주식 주문 정보가 담긴 pandas 데이터프레임.
- `start_date` (옵션): 백테스트 시작 날짜 정보. 입력하지 않으면 `df_order` 데이터프레임의 첫 날짜를 사용.
- `end_date` (옵션): 백테스트 시작 날짜 정보. 입력하지 않으면 `df_order` 데이터프레임의 마지막 날짜를 사용.
- `init_cash` (옵션): 초기보유 현금. 반드시 이름있는 인수(named parameter) 형태로 입력해야 함.

`backtest_stock_port_daily` 함수에 `df_order` 데이터프레임을 입력하여 실행하면 백테스트를 수행하고 결과를 담은 `df_result` 데이터프레임을 출력합니다.

In [3]:
dict_df_result = kq.backtest_stock_port_daily(
    df_order,
    init_cash=10_000_000,
)

[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 000660, 주문전 보유수량:      0 주문수량:     20, 매매수량:     20, 주문후 보유수량:     20
[2023-01-11] 종목: 005930, 주문전 보유수량:     10 주문수량:    -10, 매매수량:    -10, 주문후 보유수량:      0
[2023-01-18] 종목: 000660, 주문전 보유수량:     20 주문수량:    -20, 매매수량:    -20, 주문후 보유수량:      0


## 6.3 백테스트 결과 정보

포트폴리오 백테스트의 경우에는 개별주식 백테스트와 달리 백테스트 대상이 되는 모든 종목의 결과 데이터프레임 값(value)으로 가지고 해당 종목의 단축코드 문자열을 키(key)로 가지는 딕셔너리를 반환합니다.

In [4]:
dict_df_result.keys()

dict_keys(['005930', '000660', 'TOTAL'])

각 종목의 백테스트 결과 데이터프레임은 개별 종목 백테스트에서 출력되는 데이터프레임과 동일합니다.

다만 개별 종목 매매시 보유 현금을 0으로 가정하고 별도의 현금계좌로부터 융자하여 매매하는 것을 가정하므로 현금이 음수가 될 수 있습니다.

In [5]:
df_result_005930 = dict_df_result["005930"]
df_result_005930


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",0,0,0,"-555,000","-555,000","555,000","555,000",0,0,0,0,0,0
1,2023-01-03,005930,"55,400",0,0,0,10,"55,500.0000",0,0,0,0,"-555,000","555,000","554,000","-1,000",0,"-1,000","-1,000",0,"1,000"
2,2023-01-04,005930,"57,800",0,0,0,10,"55,500.0000",0,0,0,0,"-555,000","555,000","578,000","23,000",0,"23,000","23,000","23,000",0
3,2023-01-05,005930,"58,200",0,0,0,10,"55,500.0000",0,0,0,0,"-555,000","555,000","582,000","27,000",0,"27,000","27,000","27,000",0
4,2023-01-06,005930,"59,000",0,0,0,10,"55,500.0000",0,0,0,0,"-555,000","555,000","590,000","35,000",0,"35,000","35,000","35,000",0
5,2023-01-09,005930,"60,700",0,0,0,10,"55,500.0000",0,0,0,0,"-555,000","555,000","607,000","52,000",0,"52,000","52,000","52,000",0
6,2023-01-10,005930,"60,400",0,0,0,10,"55,500.0000",0,0,0,0,"-555,000","555,000","604,000","49,000",0,"49,000","49,000","52,000","3,000"
7,2023-01-11,005930,"60,500",-10,-10,"60,500",0,0.0000,0,0,0,"605,000","50,000",0,0,"50,000","50,000",0,"50,000","52,000","2,000"
8,2023-01-12,005930,"60,500",0,0,0,0,0.0000,0,0,0,0,"50,000",0,0,"50,000",0,0,"50,000","52,000","2,000"
9,2023-01-13,005930,"60,800",0,0,0,0,0.0000,0,0,0,0,"50,000",0,0,"50,000",0,0,"50,000","52,000","2,000"


In [6]:
kq.backtest_plot_stock_daily(df_result_005930)


In [7]:
df_result_000660 = dict_df_result["000660"]
df_result_000660


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,000660,"75,700",20,20,"75,700",20,"75,700.0000",0,0,0,"-1,514,000","-1,514,000","1,514,000","1,514,000",0,0,0,0,0,0
1,2023-01-03,000660,"75,600",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,512,000","-2,000",0,"-2,000","-2,000",0,"2,000"
2,2023-01-04,000660,"81,000",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,620,000","106,000",0,"106,000","106,000","106,000",0
3,2023-01-05,000660,"81,400",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,628,000","114,000",0,"114,000","114,000","114,000",0
4,2023-01-06,000660,"83,100",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,662,000","148,000",0,"148,000","148,000","148,000",0
5,2023-01-09,000660,"86,000",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,720,000","206,000",0,"206,000","206,000","206,000",0
6,2023-01-10,000660,"86,500",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,730,000","216,000",0,"216,000","216,000","216,000",0
7,2023-01-11,000660,"87,300",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,746,000","232,000",0,"232,000","232,000","232,000",0
8,2023-01-12,000660,"86,800",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,736,000","222,000",0,"222,000","222,000","232,000","10,000"
9,2023-01-13,000660,"85,700",0,0,0,20,"75,700.0000",0,0,0,0,"-1,514,000","1,514,000","1,714,000","200,000",0,"200,000","200,000","232,000","32,000"


In [8]:
kq.backtest_plot_stock_daily(df_result_000660)

실제 현금 및 수익률 등의 총합은 `"TOTAL"`이라는 키에 해당하는 백테스트 결과 데이터프레임에 저장됩니다.

In [9]:
df_result_total = dict_df_result["TOTAL"]
df_result_total

,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,TOTAL,0,0,0,0,30,0.0000,0,0,0,"-2,069,000","7,931,000","2,069,000","2,069,000","10,000,000",0,0,0,"10,000,000",0
1,2023-01-03,TOTAL,0,0,0,0,30,0.0000,0,0,0,0,"7,931,000","2,069,000","2,066,000","9,997,000",0,"-3,000","-3,000","10,000,000","3,000"
2,2023-01-04,TOTAL,0,0,0,0,30,0.0000,0,0,0,0,"7,931,000","2,069,000","2,198,000","10,129,000",0,"129,000","129,000","10,129,000",0
3,2023-01-05,TOTAL,0,0,0,0,30,0.0000,0,0,0,0,"7,931,000","2,069,000","2,210,000","10,141,000",0,"141,000","141,000","10,141,000",0
4,2023-01-06,TOTAL,0,0,0,0,30,0.0000,0,0,0,0,"7,931,000","2,069,000","2,252,000","10,183,000",0,"183,000","183,000","10,183,000",0
5,2023-01-09,TOTAL,0,0,0,0,30,0.0000,0,0,0,0,"7,931,000","2,069,000","2,327,000","10,258,000",0,"258,000","258,000","10,258,000",0
6,2023-01-10,TOTAL,0,0,0,0,30,0.0000,0,0,0,0,"7,931,000","2,069,000","2,334,000","10,265,000",0,"265,000","265,000","10,265,000",0
7,2023-01-11,TOTAL,0,0,0,0,20,0.0000,0,0,0,"605,000","8,536,000","1,514,000","1,746,000","10,282,000","50,000","232,000","282,000","10,282,000",0
8,2023-01-12,TOTAL,0,0,0,0,20,0.0000,0,0,0,0,"8,536,000","1,514,000","1,736,000","10,272,000",0,"222,000","272,000","10,282,000","10,000"
9,2023-01-13,TOTAL,0,0,0,0,20,0.0000,0,0,0,0,"8,536,000","1,514,000","1,714,000","10,250,000",0,"200,000","250,000","10,282,000","32,000"


In [10]:
kq.backtest_plot_stock_daily(df_result_total)

## 6.4 포워드테스트

포트폴리오의 경우에도 `backtest_update_stock_port_daily` 함수를 사용하여 개별주식과 마찬가지로 포워드 테스트가 가능합니다.

`backtest_update_stock_port_daily` 함수를 사용하기 위해서는 최초 백테스트를 수행할 때 다음 코드와 같이 `return_position` 인수를 `True`로 주어 현재의 주식 보유상태를 저장하는 `dict_df_position` 딕셔너리를 추가로 출력해야 합니다.

In [11]:
df_order2 = pd.DataFrame({
    "DATE": ["2023-01-02", "2023-01-02"],
    "SYMBOL": ["005930", "000660"],
    "ORDER": [10, 20]
})

dict_df_result2, dict_df_position2 = kq.backtest_stock_port_daily(
    df_order2, 
    init_cash=10_000_000, 
    return_position=True,
)


[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 000660, 주문전 보유수량:      0 주문수량:     20, 매매수량:     20, 주문후 보유수량:     20


이렇게 계산한 `dict_df_result` 및 `dict_df_position` 딕셔너리를 새로운 주문 정보와 함께 `backtest_update_stock_port_daily` 함수에 넣어주면 백테스트 결과가 추가로 갱신됩니다.

이 때 `backtest_update_stock_port_daily` 함수에는 다음과 같은 인수를 주어야 합니다.

- `symbols_and_orders`: 종목 단축코드 문자열과 매매 수량으로 이루어진 튜플의 리스트
- `date`: 매매일
- `dict_df_result`: 과거의 백테스트 결과 데이터프레임 딕셔너리
- `dict_df_position`: 과거의 백테스트 포지션 데이터프레임 딕셔너리

In [12]:
dict_df_result3, dict_df_position3 = kq.backtest_update_stock_port_daily(
    [("005930", -10)],
    "2023-01-11", 
    dict_df_result2, 
    dict_df_position2,
)


[2023-01-11] 종목: 005930, 주문전 보유수량:     10 주문수량:    -10, 매매수량:    -10, 주문후 보유수량:      0


In [13]:
dict_df_result4, dict_df_position4 = kq.backtest_update_stock_port_daily(
    [("000660", -20)],
    "2023-01-18", 
    dict_df_result3, 
    dict_df_position3,
)


[2023-01-18] 종목: 000660, 주문전 보유수량:     20 주문수량:    -20, 매매수량:    -20, 주문후 보유수량:      0


이렇게 하면 최종적으로 앞의 예제와 동일한 백테스트 결과를 얻을 수 있습니다.

In [14]:
print(
    dict_df_result["005930"].equals(dict_df_result4["005930"]),
    dict_df_result["000660"].equals(dict_df_result4["000660"]),
    dict_df_result["TOTAL"].equals(dict_df_result4["TOTAL"])
)


True True True


## 6.5 전략 함수를 사용한 백테스트

프트폴리오 백테스트 함수의 경우에도 `order` 인수에 실제 주문정보가 담긴 데이터프레임이 아닌 주문정보를 생성할 수 있는 전략 함수를 받을 수도 있습니다. 

`order` 인수로 넣을 수 있는 함수는 반드시 다음과 같은 인수를 가져야 합니다.

- `date`: datetime.date 형식의 날짜 정보
- `dict_df_result`: 백테스트 결과 데이터프레임 딕셔너리
- `dict_df_position`: 백테스트 포지션 데이터프레임 딕셔너리
- `logger`: 로거 객체

다음 예제 코드는 특정 주식들을 2023년 1월 2일에 10주식 매수하고 다음으로 찾아오는 수요일 마다 1주씩 추가 매수하는 전략을 구현한 코드입니다.

In [15]:
import datetime as dt
import logging

import pandas as pd


def trade_func(
    date: dt.date,
    dict_df_result: dict[str, pd.DataFrame],
    dict_df_position: dict[str, pd.DataFrame],
    logger: logging.Logger,
) -> list[tuple[str, int]]:
    r"""주식매매 지시함수

    주식매매 지시함수에 대한 설명

    :param dt.date date: 매매일 날짜
    :param dict[str, pd.DataFrame] dict_df_result: 매매일까지의 주문 및 체결 정보
    :param dict[str, pd.DataFrame] dict_df_position: 매매일의 주식 보유 정보
    :param logging.Logger logger: 로거
    :return list[tuple[str, int]]: 주식매매 지시
    """
    
    # 시가총액 상위 5개 주식
    symbols = [
        "005930",
        "373220",
        "000660",
        "207940",
        "005490",
    ]

    if date == dt.date(2023, 1, 2):  # 투자 시작일
        # 각 종목을 10주씩 매수
        symbols_and_orders = [
            (symbols[0], 10),
            (symbols[1], 10),
            (symbols[2], 10),
            (symbols[3], 10),
            (symbols[4], 10),
        ]
    elif date.weekday() == 1:  # 매주 화요일마다
        # 각 종목을 추가적으로 1주씩 매수
        symbols_and_orders = [
            (symbols[0], 1),
            (symbols[1], 1),
            (symbols[2], 1),
            (symbols[3], 1),
            (symbols[4], 1),
        ]
    else:
        symbols_and_orders = []

    return symbols_and_orders


In [16]:
dict_df_result5 = kq.backtest_stock_port_daily(
    trade_func,
    "2023-01-02",
    "2023-01-13",
    init_cash=1_000_000_000,
)


[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 373220, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 000660, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 207940, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 005490, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-03] 종목: 005930, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 373220, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 000660, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 207940, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 005490, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-10] 종목: 005930, 주문전 보유수량:     11 주문수량:      1, 매매수량:      1, 주문후 보유수량:     12
[2023-01-10] 종목: 373220, 주문전 보유수량:     11 주

매매 결과는 다음과 같습니다.

In [17]:
df_result5_005930 = dict_df_result5["005930"]
df_result5_005930


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,005930,"55,500",10,10,"55,500",10,"55,500.0000",0,0,0,"-555,000","-555,000","555,000","555,000",0,0,0,0,0,0
1,2023-01-03,005930,"55,400",1,1,"55,400",11,"55,490.9091",0,0,0,"-55,400","-610,400","610,400","609,400","-1,000",0,"-1,000","-1,000",0,"1,000"
2,2023-01-04,005930,"57,800",0,0,0,11,"55,490.9091",0,0,0,0,"-610,400","610,400","635,800","25,400",0,"25,400","25,400","25,400",0
3,2023-01-05,005930,"58,200",0,0,0,11,"55,490.9091",0,0,0,0,"-610,400","610,400","640,200","29,800",0,"29,800","29,800","29,800",0
4,2023-01-06,005930,"59,000",0,0,0,11,"55,490.9091",0,0,0,0,"-610,400","610,400","649,000","38,600",0,"38,600","38,600","38,600",0
5,2023-01-09,005930,"60,700",0,0,0,11,"55,490.9091",0,0,0,0,"-610,400","610,400","667,700","57,300",0,"57,300","57,300","57,300",0
6,2023-01-10,005930,"60,400",1,1,"60,400",12,"55,900.0000",0,0,0,"-60,400","-670,800","670,800","724,800","54,000",0,"54,000","54,000","57,300","3,300"
7,2023-01-11,005930,"60,500",0,0,0,12,"55,900.0000",0,0,0,0,"-670,800","670,800","726,000","55,200",0,"55,200","55,200","57,300","2,100"
8,2023-01-12,005930,"60,500",0,0,0,12,"55,900.0000",0,0,0,0,"-670,800","670,800","726,000","55,200",0,"55,200","55,200","57,300","2,100"
9,2023-01-13,005930,"60,800",0,0,0,12,"55,900.0000",0,0,0,0,"-670,800","670,800","729,600","58,800",0,"58,800","58,800","58,800",0


In [18]:
kq.backtest_plot_stock_daily(df_result5_005930)

In [19]:
df_result5_total = dict_df_result5["TOTAL"]
df_result5_total


,DATE,SYMBOL,PRICE,ORDER,QTY,TRADE_PRICE,POSITION,AVG_PRICE,FEE,TRADE_TAX,SLIPPAGE,CASHFLOW,CASH,HIST_VALUE,STOCK_VALUE,TOTAL_VALUE,REAL_PROFIT,UNREAL_PROFIT,PROFIT,HIGHWATERMARK,DRAWDOWN
0,2023-01-02,TOTAL,0,0,0,0,50,0.0000,0,0,0,"-16,762,000","983,238,000","16,762,000","16,762,000","1,000,000,000",0,0,0,"1,000,000,000",0
1,2023-01-03,TOTAL,0,0,0,0,55,0.0000,0,0,0,"-1,646,500","981,591,500","18,408,500","18,111,500","999,703,000",0,"-297,000","-297,000","1,000,000,000","297,000"
2,2023-01-04,TOTAL,0,0,0,0,55,0.0000,0,0,0,0,"981,591,500","18,408,500","18,153,300","999,744,800",0,"-255,200","-255,200","1,000,000,000","255,200"
3,2023-01-05,TOTAL,0,0,0,0,55,0.0000,0,0,0,0,"981,591,500","18,408,500","18,255,600","999,847,100",0,"-152,900","-152,900","1,000,000,000","152,900"
4,2023-01-06,TOTAL,0,0,0,0,55,0.0000,0,0,0,0,"981,591,500","18,408,500","18,481,100","1,000,072,600",0,"72,600","72,600","1,000,072,600",0
5,2023-01-09,TOTAL,0,0,0,0,55,0.0000,0,0,0,0,"981,591,500","18,408,500","18,828,700","1,000,420,200",0,"420,200","420,200","1,000,420,200",0
6,2023-01-10,TOTAL,0,0,0,0,60,0.0000,0,0,0,"-1,710,400","979,881,100","20,118,900","20,524,800","1,000,405,900",0,"405,900","405,900","1,000,420,200","14,300"
7,2023-01-11,TOTAL,0,0,0,0,60,0.0000,0,0,0,0,"979,881,100","20,118,900","20,607,600","1,000,488,700",0,"488,700","488,700","1,000,488,700",0
8,2023-01-12,TOTAL,0,0,0,0,60,0.0000,0,0,0,0,"979,881,100","20,118,900","20,727,600","1,000,608,700",0,"608,700","608,700","1,000,608,700",0
9,2023-01-13,TOTAL,0,0,0,0,60,0.0000,0,0,0,0,"979,881,100","20,118,900","20,886,000","1,000,767,100",0,"767,100","767,100","1,000,767,100",0


In [20]:
kq.backtest_plot_stock_daily(df_result5_total)

## 6.6 전략 함수를 사용한 포워드테스트

전략함수를 사용한 포트폴리오 포워드 테스트도 개별주식의 경우와 비슷한 방법으로 가능합니다.

포워드테스트를 하는 경우에는 
전략 함수 객체를 `backtest_update_stock_port_daily` 함수에 바로 넣을 수 없으므로
사용자가 전략함수를 실행하여 주문 수량을 계산한 뒤 이 값을 넣어야 합니다.

일단 전략함수를 인수로 넣고 백테스트를 수행합니다. 이 때 `return_logger` 인수를 True로 넣으면  전략함수에 넣을 로거 객체를 반환합니다.

In [21]:
dict_df_result6, dict_df_position6, logger = kq.backtest_stock_port_daily(
    trade_func,
    "2023-01-02",
    "2023-01-02",
    init_cash=1_000_000_000,
    return_position=True,
    return_logger=True,
)


[2023-01-02] 종목: 005930, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 373220, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 000660, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 207940, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10
[2023-01-02] 종목: 005490, 주문전 보유수량:      0 주문수량:     10, 매매수량:     10, 주문후 보유수량:     10


다음으로 사용자가 직접 매매수량을 계산합니다.

In [22]:
from dateutil.parser import parse

date = parse("2023-01-03")
symbols_and_orders = trade_func(
    date, 
    dict_df_result6,
    dict_df_position6, 
    logger,
)
symbols_and_orders


[('005930', 1), ('373220', 1), ('000660', 1), ('207940', 1), ('005490', 1)]

계산한 매매수량과 과거 백테스트 데이터프레임을 `backtest_update_stock_daily` 함수에 입력하여 포워드테스트를 수행합니다.

In [23]:

dict_df_result7, dict_df_position7 = kq.backtest_update_stock_port_daily(
    symbols_and_orders,
    date,
    dict_df_result6,
    dict_df_position6,
)


[2023-01-03] 종목: 005930, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 373220, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 000660, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 207940, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11
[2023-01-03] 종목: 005490, 주문전 보유수량:     10 주문수량:      1, 매매수량:      1, 주문후 보유수량:     11


앞서 수행한 백테스트와 동일한 결과가 나오는 것을 확인할 수 있습니다.

In [27]:
dict_df_result5["005930"].iloc[:2].equals(dict_df_result7["005930"])

True

같은 방식으로 매일 포워드테스트를 진행할 수 있습니다.